## CESM2 - LARGE ENSEMBLE (LENS2)

#### by Mauricio Rocha and Dr. Gustavo Marques

- In this Notebook we analyze the variation of the maximum depth of the mixed layer using Hovmoller diagrams. 
- P.S.: The notebook is under development. 

### Imports

In [ ]:
import intake
import intake_esm
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import fsspec
import cmocean
import cartopy
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pop_tools
import sys
from distributed import Client
from ncar_jobqueue import NCARCluster
sys.path.append('../functions')
import util
from cartopy.util import add_cyclic_point
from misc import get_ij
import warnings, getpass, os

<div class="alert alert-block alert-info">
<b>Note:</b> comment the following line when debugging
</div>

In [ ]:
warnings.filterwarnings("ignore")

### Local functions

### Dask workers

In [ ]:
mem_per_worker = 30 # in GB 
num_workers = 30 
cluster = NCARCluster(cores=4, processes=3, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=6:mem={mem_per_worker}GB')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

### Data Ingest

In [ ]:
%%time
catalog = intake.open_esm_datastore(
    '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
)

In [ ]:
catalog.search(component='ocn').unique('frequency')

### Let's search for variables with montly frequency

In [ ]:
cat_subset = catalog.search(component='ocn',
                            frequency='month_1',
                            variable=['XMXL']) # Maximum Mixed Layer Depth

In [ ]:
%%time
dset_dict_raw = cat_subset.to_dataset_dict()

In [ ]:
# Variables
fb=(['XMXL'])
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'ds_hist_cmip6_{fb[ifb]} = dset_dict_raw[\'ocn.historical.pop.h.cmip6.{fb[ifb]}\']'
    exec(str)
    str=f'ds_hist_smbb_{fb[ifb]} = dset_dict_raw[\'ocn.historical.pop.h.smbb.{fb[ifb]}\']'
    exec(str)
    str=f'ds_hist_{fb[ifb]} = xr.concat([ds_hist_cmip6_{fb[ifb]},ds_hist_smbb_{fb[ifb]}], dim=\'member_id\',data_vars=\'minimal\',coords=\'minimal\',compat=\'override\')'
    exec(str)
    str=f'mem=ds_hist_{fb[ifb]}.{fb[ifb]}.nbytes*1e-12 # in TB'
    exec(str)
    print(f'Memory: {mem} TB')
    str=f'del ds_hist_cmip6_{fb[ifb]}; del ds_hist_smbb_{fb[ifb]}'; exec(str) 
print(f'Done!')

### Import the POP grid

If you choose the ocean component of LENS2, you will need to import the POP grid. For the other components, you can use the emsemble's own grid. 

In ds, TLONG and TLAT have missing values (NaNs), so we need to override them with the values from pop_grid, which does not have missing values.

In [ ]:
# Read the pop 1 deg grid from pop_tools
# We will use variables TLONG and TLAT
pop_grid = pop_tools.get_grid('POP_gx1v7')
for ifb in range(0,len(fb)):
    print(f'Variable: {fb[ifb]}')
    str=f'ds_hist_{fb[ifb]}[\'TLONG\'] = pop_grid.TLONG'     # Longitud
    exec(str)
    str=f'ds_hist_{fb[ifb]}[\'TLAT\'] = pop_grid.TLAT'       # Latitudes
    exec(str)
    str=f'ds_hist_{fb[ifb]}[\'TLONG\'] = pop_grid.TLONG'     # Longitud
    exec(str)
    str=f'ds_hist_{fb[ifb]}[\'TLAT\'] = pop_grid.TLAT'       # Latitudes
    exec(str)

In [ ]:
ds_var_1231 = ds_hist_XMXL.XMXL.sel(member_id=['r1i1231p1f1','r2i1231p1f1','r3i1231p1f1','r4i1231p1f1','r5i1231p1f1','r6i1231p1f1','r7i1231p1f1','r8i1231p1f1','r9i1231p1f1','r10i1231p1f1']).mean(dim='time')#.plot()
ds_var_1231 = ds_var_1231*-0.01 # cm to -m
plt.figure(figsize=(10,6));
ax = plt.axes(projection=ccrs.Robinson());
pc = ds_var_1231.mean(dim='member_id').plot.pcolormesh(ax=ax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmocean.cm.thermal,
                    x='TLONG',
                    y='TLAT',
                    vmin=-700,
                    vmax=0,
                    cbar_kwargs={'orientation': 'horizontal'})                                    
ax.gridlines(draw_labels=True);
ax.coastlines()
ax.gridlines();
plt.title('i1231p1f1')
del ds_var_1231

### Centralize the South Atlantic 
Need to combine the domain in the east/west direction to centralize the South Atlantic

In [ ]:
ds_hist_XMXL=ds_hist_XMXL.XMXL
ds_hist_XMXL=ds_hist_XMXL*-0.01 # cm to -m

In [ ]:
ilat, flat = 280, 384
ilon1, flon1, ilon2, flon2 = 280, 320, 0, 57
na_ds_XMXL=xr.combine_nested([[ds_hist_XMXL.isel(nlat = slice(ilat,flat),nlon = slice(ilon1,flon1)),ds_hist_XMXL.isel(nlat = slice(ilat,flat),nlon = slice(ilon2,flon2))]],concat_dim=['nlat','nlon'])
na_ds_XMXL.coords['TLONG'] = (na_ds_XMXL.coords['TLONG'] + 180) % 360 - 180 # change the longitudes: -180 0 180

In [ ]:
# simple check
na_ds_XMXL.isel(time=2, member_id=0).plot()

In [ ]:
%%time
plt.figure(figsize=(10,6));
ax = plt.axes(projection=ccrs.Robinson());
pc = na_ds_XMXL.isel(member_id=0).mean(dim='time').plot.pcolormesh(ax=ax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmocean.cm.balance,
                    x='TLONG',
                    y='TLAT',
                    #vmin=10,
                    #vmax=30,
                    cbar_kwargs={"orientation": "horizontal"})                                    
ax.gridlines(draw_labels=True);
ax.coastlines()
ax.gridlines();

In [ ]:
na_ds_XMXL.isel(member_id=0).mean(dim='time').sel(nlat=50,method='nearest')

In [ ]:
%%time
# Compute a value to define a region of maximum in each section. 
Z1 = na_ds_XMXL.mean(dim='member_id').isel(nlat=67)
Z1_max=np.sort(np.max(Z1, axis=1).values)[-4] # I chose a region of maximum, selecting the fourth maximum value of all the years
print(round(Z1_max,1))

Z2 = na_ds_XMXL.mean(dim='member_id').isel(nlon=29)
Z2_max=np.sort(np.max(Z2, axis=1).values)[-4]
print(round(Z2_max,1))

Z3 = na_ds_XMXL.mean(dim='member_id').isel(nlat=95)
Z3_max=np.sort(np.max(Z3, axis=1).values)[-4]
print(round(Z3_max,1))

Z4 = na_ds_XMXL.mean(dim='member_id').isel(nlon=85)
Z4_max=np.sort(np.max(Z4, axis=1).values)[-4]
print(round(Z4_max,1))

X = np.meshgrid(na_ds_XMXL['TLONG'],na_ds_XMXL['time'])
Y = np.meshgrid(na_ds_XMXL['TLAT'],na_ds_XMXL['time'])

In [ ]:
np.max(na_ds_XMXL.isel(nlat=67,nlon=30).mean(dim='member_id'),axis=0).plot()

In [ ]:
mn=-100
mx=0

In [ ]:
%%time
# Plot
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=[17, 5])
ax1.contourf(X, Y, Z1, 20, vmin=mn, vmax=mx, cmap="RdBu_r")
ax2.contourf(X, Y, Z2, 20, vmin=mn, vmax=mx, cmap="RdBu_r")
ax3.contourf(X, Y, Z3, 20, vmin=mn, vmax=mx, cmap="RdBu_r")

im = ax4.contourf(X, Y, Z4, 20, vmin=mn, vmax=mx, cmap="RdBu_r")
axins = inset_axes(ax4, width='5%', height='100%', loc='lower left', bbox_to_anchor=(1.05, 0., 1, 1), bbox_transform=ax4.transAxes, borderpad=0)
fig.colorbar(im, cax=axins, ticks=range(mn, mx, 2),label='Sv')

# Subplots
fmax=10
ax1.contour(X, Y, Z1, levels = [0], colors=('k',), linestyles=('--',), linewidths=(1,)); CS=ax1.contour(X, Y, Z1, levels = [-7.7], colors=('k',), linestyles=('-',), linewidths=(2,))
ax1.clabel(CS, inline=True, fontsize=fmax); del CS

ax2.contour(X, Y, Z2, levels = [0], colors=('k',), linestyles=('--',), linewidths=(1,)); CS=ax2.contour(X, Y, Z2, levels = [-21.4], colors=('k',), linestyles=('-',), linewidths=(2,))
ax2.clabel(CS, inline=True, fontsize=fmax); del CS

ax3.contour(X, Y, Z3, levels = [0], colors=('k',), linestyles=('--',), linewidths=(1,)); CS=ax3.contour(X, Y, Z3, levels = [-9.1], colors=('k',), linestyles=('-',), linewidths=(2,))
ax3.clabel(CS, inline=True, fontsize=fmax); del CS

ax4.contour(X, Y, Z4, levels = [0], colors=('k',), linestyles=('--',), linewidths=(1,)); CS=ax4.contour(X, Y, Z4, levels = [-8.7], colors=('k',), linestyles=('-',), linewidths=(2,))
ax4.clabel(CS, inline=True, fontsize=fmax); del CS

#Labels
ax1.set_title('-33.008255S (SAMOC-34.5S)'); ax2.set_title('-10.9519205S (11S)'); ax3.set_title('26.273556N (RAPID)'); ax4.set_title('39.953373N (AMOC Maxima)')
ax1.set_xlabel('Depth [m]'); ax2.set_xlabel('Depth [m]'); ax3.set_xlabel('Depth [m]'); ax4.set_xlabel('Depth [m]')
ax1.set_ylabel('Time [Years]')
ax1.grid(color='k', linestyle='-', linewidth=0.7); ax2.grid(color='k', linestyle='-', linewidth=0.7); ax3.grid(color='k', linestyle='-', linewidth=0.7); ax4.grid(color='k', linestyle='-', linewidth=0.7)
plt.show()